In [26]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

import numpy as np
import pandas as pd

from PIL import Image

from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch.nn as nn
from torchvision import models

In [28]:
class ImageDataset(Dataset):
    def __init__(self, path: str, parameters: pd.DataFrame, transform=None) -> None:
        self.image_paths = np.array([path + filename for filename in parameters['filename'].to_numpy()])
        self.labels = parameters['epsilon'].to_numpy()
        self.transform = transform

    def __getitem__(self, inx):
        image_path = self.image_paths[inx]
        label_float_epsilon = self.labels[inx]
        target = np.zeros(shape=(1000,))
        target[int(label_float_epsilon*1000)] = 1
        image = Image.open(image_path)
        image = np.array(image)
        # repeat grayscale value three times for all RGB channels
        image = np.repeat(image[..., np.newaxis], 3, -1)
        if self.transform:
            image = self.transform(image)
        return image, torch.from_numpy(target)

    def __len__(self):
        return len(self.image_paths)

In [29]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [30]:
def test(model, test_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for data, labels in tqdm(test_loader):
            images = data.cuda()
            targets = labels.cuda()
            outputs = model(images)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [31]:
def cyclic_MAE_test(output, target):
    return min(abs(output-target), min(abs(output-target+1), abs(output-target-1)))

In [32]:
def save_results(outputs, targets, filename: str):
    epsilons = []
    predicted = []
    differences = []
    for out, tar in zip(outputs, targets):

        epsilons.append(tar.index(1)/1000)
        predicted.append(out.index(1)/1000)
        differences.append(cyclic_MAE_test(out.index(1)/1000, tar.index(1)/1000))
    results_df = pd.DataFrame(list(zip(epsilons, predicted, differences)),
                  columns =['epsilon', 'predicted', 'loss'])
    results_df.to_csv(filename, index=False)
    return results_df

In [41]:
main_dir = "../../data/generated/"

all_df = pd.read_csv(main_dir + "all_test.csv", index_col=False)
pure_df = pd.read_csv(main_dir + "pure_test.csv", index_col=False)
average_df = pd.read_csv(main_dir + "average_test.csv", index_col=False)
bubble_df = pd.read_csv(main_dir + "bubble_test.csv", index_col=False)
blackbox_df = pd.read_csv(main_dir + "blackbox_test.csv", index_col=False)
pizza_df = pd.read_csv(main_dir + "pizza_test.csv", index_col=False)
perlin_average = pd.read_csv(main_dir + "perlin_ceramic_test/parameters.csv", index_col=False)
perlin_average_moved = pd.read_csv(main_dir + "perlin_ceramic_moved_test/parameters.csv", index_col=False)

### pure test ###

In [10]:
pure_df["filename"] = "pure/" + pure_df["filename"]

test_dataset = ImageDataset(main_dir, pure_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [38]:
model = models.resnet18(pretrained=True)

classifier = nn.Sequential(
    nn.Linear(512, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.ReLU(),
    nn.Linear(512, 1000)
)

model.fc = classifier

c:\Users\janko\OneDrive\Pulpit\Artificial intelligence\Magisterka\topography-public\venv_torch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\janko\OneDrive\Pulpit\Artificial intelligence\Magisterka\topography-public\venv_torch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [42]:
model.load_state_dict(torch.load('../../models/final_model_best.pth'))
model.cuda()
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [16]:
outputs, targets = test(model, test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "steel_model-pure_noise_results.csv") 
results['loss'].mean() 

100%|██████████| 94/94 [00:33<00:00,  2.83it/s]


0.006583666666666667

In [45]:
# pure_df["filename"] = "pure/" + pure_df["filename"]
# average_df["filename"] = "average/ceramic/" + average_df["filename"]
# perlin_average["filename"] = "perlin_ceramic_test/" + perlin_average["filename"]
# perlin_average_moved["filename"] = "perlin_ceramic_moved_test/" + perlin_average_moved["filename"]

test_dataset = ImageDataset(main_dir, perlin_average, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

outputs, targets = test(model, test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "ceramic_model-perlin_average_results.csv") 
results['loss'].mean() 

100%|██████████| 469/469 [01:31<00:00,  5.11it/s]


0.017798799999999997

In [20]:
pizza_df["filename"] = "pizza/" + pizza_df["filename"]

test_dataset = ImageDataset(main_dir, pizza_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

outputs, targets = test(model, test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "steel_model-triangle_noise_results.csv") 
results['loss'].mean()

100%|██████████| 94/94 [00:33<00:00,  2.77it/s]


0.007040000000000001

In [21]:
bubble_df["filename"] = "bubble/" + bubble_df["filename"]

test_dataset = ImageDataset(main_dir, bubble_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

outputs, targets = test(model, test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "steel_model-bubble_noise_results.csv") 
results['loss'].mean()

100%|██████████| 94/94 [00:33<00:00,  2.84it/s]


0.007877666666666668

In [22]:
blackbox_df["filename"] = "blackbox/" + blackbox_df["filename"]

test_dataset = ImageDataset(main_dir, blackbox_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

outputs, targets = test(model, test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "steel_model-blackbox_noise_results.csv") 
results['loss'].mean()

100%|██████████| 94/94 [00:32<00:00,  2.86it/s]


0.034106

In [25]:
all_df["filename"] = "all/ceramic/" + all_df["filename"]

test_dataset = ImageDataset(main_dir, all_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

outputs, targets = test(model, test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "steel_model-all_noise_results.csv") 
results['loss'].mean()

100%|██████████| 94/94 [00:19<00:00,  4.85it/s]


0.014125666666666668